Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 1.73 ms, sys: 2.86 ms, total: 4.59 ms
Wall time: 10.9 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.209/10352/1 Dashboard: http://10.20.0.209:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 176 ms, sys: 15.7 ms, total: 192 ms
Wall time: 1.79 s


[1.274429621968879,
 1.696621710036045,
 1.3705493770766028,
 1.936889352493175,
 1.4061116505248854,
 1.2226811143829426,
 1.0251145215479265,
 1.5040982955217,
 1.8975738223809262,
 1.3349958676453788,
 1.8500590181071708,
 1.5109272895093024,
 1.397267377661373,
 1.593619667967198,
 1.9298244892874168,
 1.012135957941978,
 1.5766701674986416,
 1.4329921336853562,
 1.0027078167225019,
 1.232381630652124,
 1.9547242384298906,
 1.4333149716439249,
 1.4388067908036875,
 1.4201293190468327,
 1.3758917133361446,
 1.558538287261845,
 1.7276492678369082,
 1.7130234168176441,
 1.8608886331279157,
 1.4911186423775582,
 1.306585361488862,
 1.351075484171884,
 1.3373961923800874,
 1.240484668279231]

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.148330064195966,
 1.5198911634206262,
 1.2356176523180902,
 1.3018042831935066,
 1.1732706235995611,
 1.8900661819105524,
 1.824123174732661,
 1.2631285644241774,
 1.9985280708496118,
 1.0731630413341957,
 1.3756319413200777,
 1.463244345472323,
 1.3902538201038444,
 1.806751491160565,
 1.704279457240849,
 1.5735390736727926,
 1.0081802885091404,
 1.7161782706583124,
 1.5669093843352302,
 1.3538534575928076,
 1.636512986893182,
 1.3471077456996174,
 1.6411875896751165,
 1.553163943317331,
 1.3005642351485962,
 1.191591945856827,
 1.0617004573687625,
 1.6807737903003601,
 1.4596898600277852,
 1.3215691103109974,
 1.5637224609111453,
 1.1603582668854586,
 1.2653919513733896,
 1.5385983935190168,
 1.8428697208882632,
 1.1223803713140466,
 1.7691082302825663,
 1.0548437244684536,
 1.390256716027042,
 1.5775991833075822,
 1.7051284423701318,
 1.179954768681453,
 1.0358913994854086,
 1.4756525781680359,
 1.331035196619552,
 1.1375853513446301,
 1.483783866234392,
 1.9265383502110354,
 1.4

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 166 ms, sys: 19.3 ms, total: 185 ms
Wall time: 1.99 s
